# 03 - Multi Layer Network

In this notebook we try stacking multiple hidden layer inside our rnn cell.

In [1]:
import tools.processing as pre
import re

# use less text for now to avoid memory error
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")
vocab = pre.Vocabulary(text)
# double \\n to avoid null error in tensorboard projection
text = text.replace("\n", " \\n ")
# remove extra spacing
tokens = re.sub( " +", " ", text).split(" ")[:-1]

TIMESTEPS = 16

str_data, str_labels = pre.create_data_label_pairs(tokens, TIMESTEPS)

print( list( zip(str_data, str_labels) )[:5] )

[(['as', 'real', 'as', 'it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another', 'calculated', 'schemes', '\\n'], 'to'), (['real', 'as', 'it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another', 'calculated', 'schemes', '\\n', 'to'], 'get'), (['as', 'it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another', 'calculated', 'schemes', '\\n', 'to', 'get'], 'us'), (['it', 'seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another', 'calculated', 'schemes', '\\n', 'to', 'get', 'us'], 'locked'), (['seems', 'the', 'american', 'dream', '\\n', "ain't", 'nothing', 'but', 'another', 'calculated', 'schemes', '\\n', 'to', 'get', 'us', 'locked'], 'up')]


In [2]:
import tools.training as tr

encoder = tr.OneHotWordEncoder("1-Hot-Word-Encoding", vocab.word2index)
decoder = tr.OneHotWordDecoder("1-Hot-Word-Decoding", vocab.index2word, temperature=0.8)

data = encoder.encode( str_data )
labels = encoder.encode_labels( str_labels )

del str_labels
del str_data
del tokens

In [3]:
import tools.architectures as nn
import tensorflow as tf

class SimpleMultiLayerRNN(nn.Trainable):
    def __init__(self, name):
        super().__init__(name)

    def build(self, num_layers, hidden_layer_size, vocab_size, time_steps, l2_reg=0.0):
        self.time_steps = time_steps
        self.vocab_size = vocab_size

        self.X = tf.placeholder(tf.float32, shape=[None, time_steps, vocab_size], name="data")
        self.Y = tf.placeholder(tf.int16, shape=[None, vocab_size], name="labels")

        self.X = tf.placeholder(tf.float32, shape=[None, time_steps, vocab_size], name="data")
        
        with tf.variable_scope(self.name, reuse=tf.AUTO_REUSE):

            self.stacked_cells = nn.lstm_layer(num_layers, hidden_layer_size)

            self.outputs, self.states = tf.nn.dynamic_rnn(
                    self.stacked_cells, self.X, dtype=tf.float32)
            
            self.last_rnn_output = self.states[num_layers - 1][1]

            self.final_output, W_out, b_out = nn.full_layer(self.last_rnn_output, vocab_size)

            self.weights.append(W_out)
            self.biases.append(b_out)

            self.softmax = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.final_output,
                    labels=self.Y)
            self.cross_entropy_loss = tf.reduce_mean(self.softmax)

            self.loss = self.cross_entropy_loss

            self.optimizer = tf.train.AdamOptimizer()
            self.train_step= self.optimizer.minimize(self.loss)

            self.correct_prediction = tf.equal(tf.argmax(self.Y,1), tf.argmax(self.final_output, 1))
            self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))*100

In [4]:
import tools.processing as pre
import tools.architectures as nn

NUM_LAYERS = 2
HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = vocab.get_size()

EPOCHS = 30
BATCH_SIZE = 256

rnn = SimpleMultiLayerRNN(name = "multi-pac")
rnn.build(NUM_LAYERS, HIDDEN_LAYER_SIZE, VOCAB_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, seed_text: tr.sample( seed_text, trainable, encoder, decoder, length=20)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE, log_dir="logs/03-multi-layer-lstm", retrain=False)

Building model from scratch! 
 Saving into: 'logs/03-multi-layer-lstm-v2'


Epoch 1/30
Loss:    	 5.963024139404297
Accuracy:	 10.942182540893555
------Sampling----------
seed: 
as real as it seems the american dream
    ain't nothing but another calculated schemes
to get us locked up shot up back in chains
    to deny us of the future rob our names
kept my history of mystery but now i see
    the american dream wasn't meant for me
cause lady liberty is a hypocrite she lied to me
promised me freedom education equality
    never gave me nothing but slavery
and now look at how dangerous you made me
-
result: 
as real as it seems the american dream
    ain't nothing but another calculated schemes
to get us locked up shot up back in chains
    to deny us of the future rob our names
kept my history of mystery but now i see
    the american dream wasn't meant for me
cause lady liberty is a hypocrite she lied to me
promised me freedom education equality
    never gave me nothing but slavery
a

In [7]:
decoder.temperature = 1

sampler = lambda trainable, seed_text: tr.sample( seed_text, trainable, encoder, decoder, length=50)
sampler(rnn, "killin people left and right \n use a gun cool homie \n that is right")

------Sampling----------
seed: 
killin people left and right 
 use a gun cool homie 
 that is right
-
result: 
killin people left and right 
 use a gun cool homie 
 that is right so i bust it alone is know think i got up y'all mine 
 out this thing a clip for can on your got up 
 up to hoodlum who sell can from to get em as it 
 niggaz was then niggaz 6 
 knowledge my head 
 2pac


# Problem

How can we work out proper features from the text?

Just because a line does not match 100% with the original one that doesn't mean that it is bad

We shall map our 1-hot-encoded vector to a proper feature space using an embedding!